In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the Boston Housing dataset
from sklearn.datasets import fetch_california_housing
boston = fetch_california_housing()

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(boston.data)
y = boston.target.reshape(-1, 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a list of architectures to compare
architectures = [
    [64],
    [128],
    [64, 64],
    [128, 64],
]

# Define a list of activation functions to compare
activation_functions = [
    'relu',
    'tanh',
    'sigmoid',
]

# Define a list of learning rates to compare
learning_rates = [
    0.001,
    0.01,
    0.1,
]

# Train and evaluate each model
for architecture in architectures:
    for activation_function in activation_functions:
        for learning_rate in learning_rates:
            # Define the neural network
            model = tf.keras.models.Sequential([
              tf.keras.layers.Dense(architecture[0], activation=activation_function, input_shape=[X.shape[1]]),
            ])
            for layer in architecture[1:]:
              model.add(tf.keras.layers.Dense(layer, activation=activation_function))
            model.add(tf.keras.layers.Dense(1))

            # Compile the model
            model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

            # Train the model
            history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=0)

            # Print the model's validation loss
            print(f'Architecture: {architecture}, Activation Function: {activation_function}, Learning Rate: {learning_rate}')
            print(f'Validation Loss: {history.history["val_loss"][-1]:.4f}\n')

# Predict the test set using the best model
best_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', input_shape=[X.shape[1]]),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1)
])
best_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
best_history = best_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=0)
y_pred = best_model.predict(X_test)

# Plot the loss over time for the best model
plt.plot(best_history.history['loss'], label='Training Loss')
plt.plot(best_history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

# Plot the actual vs. predicted values for the best model
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()
